<a href="https://colab.research.google.com/github/matherialist/dota-2-win-probability-prediction/blob/master/DOTA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import datetime

In [0]:
features = pd.read_csv('https://d3c33hcgiwev3.cloudfront.net/_47ea8666d262362a0b41164cb497658e_features.zip?Expires=1576195200&Signature=JsDvxiIFoNW7EpdBeZ2pEVrr3KHMETtQlmAuuD-~za0YRU83JlSjSZ6fTeP6pHif~8Iv5KQC0ieVZboGmmC1xRvwHie2JafzoNtMsZl-X~SuWdaZ6hN~z~nbbquhwHYSteUUaaHvo3mdYbodhlbk2zqx3CapEDFIPebqz0azVBY_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A', compression='zip', index_col='match_id')

In [3]:
features.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_hero,r2_level,r2_xp,r2_gold,r2_lh,r2_kills,r2_deaths,r2_items,r3_hero,r3_level,r3_xp,r3_gold,r3_lh,r3_kills,r3_deaths,r3_items,r4_hero,r4_level,r4_xp,r4_gold,r4_lh,r4_kills,r4_deaths,r4_items,r5_hero,r5_level,r5_xp,r5_gold,r5_lh,r5_kills,...,d4_xp,d4_gold,d4_lh,d4_kills,d4_deaths,d4_items,d5_hero,d5_level,d5_xp,d5_gold,d5_lh,d5_kills,d5_deaths,d5_items,first_blood_time,first_blood_team,first_blood_player1,first_blood_player2,radiant_bottle_time,radiant_courier_time,radiant_flying_courier_time,radiant_tpscroll_count,radiant_boots_count,radiant_ward_observer_count,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,67,3,842,991,10,0,0,4,29,5,1909,1143,10,0,0,8,20,3,757,741,6,0,0,7,105,3,732,658,4,0,...,742,500,2,0,0,8,84,3,958,1003,3,1,0,9,7.0,1.0,9.0,NaN,134.0,-80.0,244.0,2,2,2,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,49,4,1596,993,10,0,1,7,67,4,1506,1502,18,1,0,7,37,3,669,631,7,0,0,7,26,2,415,539,1,0,...,440,583,0,0,0,7,12,4,1470,1622,24,0,0,9,54.0,1.0,7.0,NaN,173.0,-80.0,NaN,2,0,2,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,98,3,1314,775,6,0,0,6,20,3,1297,909,0,1,0,6,27,5,2360,2096,26,1,1,6,4,3,1395,1627,27,0,...,1319,667,4,0,0,7,80,3,1350,1512,25,0,0,7,224.0,0.0,3.0,NaN,63.0,-82.0,NaN,2,5,2,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,30,2,539,539,1,0,0,6,75,5,2037,1139,15,0,0,6,37,2,591,499,0,0,0,6,41,3,712,1075,12,0,...,674,537,1,0,0,7,20,2,510,499,0,0,0,7,NaN,NaN,NaN,NaN,208.0,-75.0,NaN,0,3,2,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,27,2,629,552,0,0,1,7,30,3,884,927,0,1,0,8,72,3,925,1439,16,1,0,11,93,4,1482,880,7,0,...,1815,1275,18,0,0,8,8,4,1119,904,6,0,1,7,-21.0,1.0,6.0,NaN,166.0,-81.0,181.0,1,4,2,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0,2449,0,4,1974,3,63


In [0]:
y_train = features['radiant_win']
X_train = features.loc[:, features.columns != 'radiant_win']

In [0]:
features_test = pd.read_csv('https://d3c33hcgiwev3.cloudfront.net/_6866d458468373e164b1e859f3103826_features_test.zip?Expires=1576195200&Signature=AkH0bulkEwmdyJFfiXfsKYX2Zaod23QaH-DNg9cgKsagKbilTeTFHBX4~fYKbeZCW3MJQrlKmnuGpoOMYITRUk9nFwhNCFaQbclzq6iUvSSv4ymO5AwOGvOagOgxfRTnRbYQUNPT~vTiOp~BbWJ3ypGvFW6bB2DJO3k2v0MfbhA_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A', compression='zip', index_col='match_id')

In [0]:
features_test.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_hero,r2_level,r2_xp,r2_gold,r2_lh,r2_kills,r2_deaths,r2_items,r3_hero,r3_level,r3_xp,r3_gold,r3_lh,r3_kills,r3_deaths,r3_items,r4_hero,r4_level,r4_xp,r4_gold,r4_lh,r4_kills,r4_deaths,r4_items,r5_hero,r5_level,r5_xp,r5_gold,r5_lh,r5_kills,...,d3_lh,d3_kills,d3_deaths,d3_items,d4_hero,d4_level,d4_xp,d4_gold,d4_lh,d4_kills,d4_deaths,d4_items,d5_hero,d5_level,d5_xp,d5_gold,d5_lh,d5_kills,d5_deaths,d5_items,first_blood_time,first_blood_team,first_blood_player1,first_blood_player2,radiant_bottle_time,radiant_courier_time,radiant_flying_courier_time,radiant_tpscroll_count,radiant_boots_count,radiant_ward_observer_count,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6,1430287923,0,93,4,1103,1089,8,0,1,9,102,3,1183,963,9,0,2,9,52,5,2289,2043,20,2,0,6,53,3,917,1006,14,0,0,6,57,3,1344,933,3,0,...,2,1,0,9,29,3,1091,918,6,0,1,8,39,5,1674,1048,12,0,1,7,1.0,0.0,2.0,NaN,72.0,-82.0,NaN,2,2,3,0,12.0,247.0,-86.0,272.0,3,4,2,0,118.0
7,1430293357,1,20,2,556,570,1,0,0,9,6,4,1194,1386,21,0,0,7,65,4,1374,1064,13,0,1,8,92,2,774,737,0,1,0,8,43,5,2254,1787,24,0,...,19,0,0,14,99,3,1075,718,3,0,1,7,5,4,1130,994,1,1,0,10,84.0,1.0,9.0,NaN,138.0,-82.0,233.0,2,6,2,2,-29.0,168.0,-54.0,NaN,3,2,2,1,16.0
10,1430301774,1,112,2,751,808,1,0,0,13,26,2,421,569,3,0,0,6,29,3,960,896,3,1,2,6,11,5,1877,1511,23,0,0,8,41,4,1581,1593,26,0,...,12,0,0,8,49,5,2434,1917,22,0,0,9,54,4,1372,2005,24,2,1,11,168.0,1.0,9.0,NaN,33.0,-88.0,NaN,5,4,3,1,-22.0,46.0,-87.0,186.0,1,3,3,0,-34.0
13,1430323933,1,27,3,708,903,1,1,1,11,91,2,672,901,6,0,0,8,74,5,2141,1626,28,0,0,7,52,4,1579,2169,28,2,0,7,2,5,1960,1668,27,0,...,22,0,0,7,62,2,536,692,0,1,1,6,49,4,1815,1424,18,0,0,5,55.0,0.0,3.0,NaN,208.0,-78.0,180.0,4,2,2,2,-49.0,30.0,-89.0,210.0,3,4,2,1,-26.0
16,1430331112,1,39,4,1259,661,4,0,0,9,93,5,1703,964,10,0,1,6,30,3,1099,942,2,1,0,6,85,3,1285,1013,10,0,0,8,102,2,484,609,4,0,...,19,0,0,10,3,3,623,520,1,0,0,10,96,5,1884,1409,15,0,1,11,285.0,1.0,5.0,NaN,225.0,-76.0,NaN,1,3,2,0,36.0,180.0,-86.0,180.0,1,3,2,1,-33.0


**Подход 1: градиентный бустинг "в лоб"**

In [0]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold, cross_val_score

In [0]:
X_train = features.drop(['duration', 'radiant_win', 'tower_status_radiant', 
                            'tower_status_dire', 'barracks_status_radiant', 
                            'barracks_status_dire'], axis=1)

In [0]:
X_train.shape

(97230, 102)

Определим признаки, содержащие пропуски.

In [0]:
X_train.columns[X_train.isna().any()].tolist()

['first_blood_time',
 'first_blood_team',
 'first_blood_player1',
 'first_blood_player2',
 'radiant_bottle_time',
 'radiant_courier_time',
 'radiant_flying_courier_time',
 'radiant_first_ward_time',
 'dire_bottle_time',
 'dire_courier_time',
 'dire_flying_courier_time',
 'dire_first_ward_time']

Если событие "первая кровь" не успело произойти за первые 5 минут, то признаки first_blood_time, first_blood_team, first_blood_player1, first_blood_player2 принимают пропущенное значение

In [0]:
X_train.fillna(0, inplace=True)

In [0]:
X_train.columns[X_train.isna().any()].tolist()

[]

Целевая переменная - radiant_win

In [0]:
y_train = features['radiant_win']

In [0]:
kf = KFold(n_splits=5, shuffle=True)

for n in [10, 20 , 30, 40, 50, 100]:
    clf = GradientBoostingClassifier(n_estimators=n, verbose=True)
    start_time = datetime.datetime.now()
    score = cross_val_score(clf, X_train, y_train, cv=kf, scoring="roc_auc", n_jobs=-1).mean()

    print('n_estimators =', n)
    print('Score:', score)
    print('Time elapsed:', datetime.datetime.now() - start_time)
    print()

n_estimators = 10
Score: 0.6644723421723491
Time elapsed: 0:00:25.247362

n_estimators = 20
Score: 0.6820569226035162
Time elapsed: 0:00:45.669416

n_estimators = 30
Score: 0.6898865269074778
Time elapsed: 0:01:05.856615

n_estimators = 40
Score: 0.6942058064933108
Time elapsed: 0:01:26.839650

n_estimators = 50
Score: 0.697810155255931
Time elapsed: 0:01:47.103974

n_estimators = 100
Score: 0.7068091429057917
Time elapsed: 0:03:31.404442



Для градиентного бустинга с 30 деревьями кросс-валидация длилась 0:01:05.856615, значение ROC AUC при этом 0.6899. 
Есть смысл увеличивать число деревьев, т.к. это ведёт к увеличению метрики ROC AUC.

**Подход 2: логистическая регрессия**

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [0]:
X_train = features.drop(['duration', 'radiant_win', 'tower_status_radiant', 
                            'tower_status_dire', 'barracks_status_radiant', 
                            'barracks_status_dire'], axis=1)
X_train.fillna(0, inplace=True)

In [0]:
y_train = features['radiant_win']

In [0]:
scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), index=X_train.index, columns=X_train.columns)

In [20]:
kf = KFold(n_splits=5, shuffle=True)

for i in range(-5, 5):
    C = 10.0 ** i
    clf = LogisticRegression(penalty='l2', C=C, verbose=True)
    start_time = datetime.datetime.now()
    score = cross_val_score(clf, X_train, y_train, cv=kf, scoring="roc_auc", n_jobs=-1).mean()

    print('C =', C)
    print('Score:', score)
    print('Time elapsed:', datetime.datetime.now() - start_time)
    print()

C = 1e-05
Score: 0.6951203930965234
Time elapsed: 0:00:04.398966

C = 0.0001
Score: 0.7113197916153771
Time elapsed: 0:00:05.981815

C = 0.001
Score: 0.7163510937876062
Time elapsed: 0:00:09.334101

C = 0.01
Score: 0.7163585184650173
Time elapsed: 0:00:12.308196

C = 0.1
Score: 0.7165891758956429
Time elapsed: 0:00:13.276702

C = 1.0
Score: 0.7163025288197954
Time elapsed: 0:00:12.894827

C = 10.0
Score: 0.7164020195941407
Time elapsed: 0:00:13.112117

C = 100.0
Score: 0.7163863410507656
Time elapsed: 0:00:12.884006

C = 1000.0
Score: 0.7164502687881228
Time elapsed: 0:00:12.942883

C = 10000.0
Score: 0.7163485320147955
Time elapsed: 0:00:12.987991

